In [ ]:
import csv

import numpy as np
import pandas as pd
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow import keras

from functions.image import get_filtered_satellite_float32

In [ ]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)
block_size = 30


def get_image(coord):
    return get_filtered_satellite_float32(coord, block_size, img_data, dataset)

In [ ]:
with open('training/data_bin.csv', 'r') as f:
    reader = csv.DictReader(f)
    labeled_coords = [r for r in reader]
print("labeled dataset contains {} points".format(len(labeled_coords)))

In [ ]:
X = list(map(get_image, labeled_coords))
Y = [int(l['windmill']) for l in labeled_coords]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=10)

In [ ]:
model.evaluate(X_test, Y_test, verbose=2)

In [ ]:
left_top_coord = (2.79300,51.68376) # enter coordinate here
#img_h, img_w = (750, 500)
img_h, img_w = (1750, 1500)
left_top_x, left_top_y = dataset.index(left_top_coord[0], left_top_coord[1])
needles = []
step_size = 8
raw_coords = []

x_count = ((img_h - block_size) // step_size)
for xx in range(left_top_x + block_size // 2, left_top_x + img_w - block_size // 2, step_size):
    for yy in range(left_top_y + block_size // 2, left_top_y + img_h - block_size // 2, step_size):
        raw_coords.append((xx,yy))
        cc = dataset.transform * (yy, xx)
        needles.append({'lon': cc[0], 'lat': cc[1]})

needle_data = np.stack([get_image(coord) for coord in needles])
results = model.predict(needle_data)


In [ ]:
from matplotlib import pyplot as plt

search_space = np.copy(img_data[left_top_x:left_top_x + img_w, left_top_y:left_top_y + img_h])
highlight = np.max(search_space)/2

found = 0
previous_i = 0
found_i = []
coords = []

final_results = []

for i, result in enumerate(results):
    if result[1] > result[0]:
        if (i - previous_i <= 2):
            continue
        if (i - x_count in found_i or (i - (x_count * 2)) in found_i or (i + 1 - x_count) in found_i):
            continue
        found += 1
        previous_i = i
        found_i.append(i)
        coord = needles[i]
        xx, yy = dataset.index(coord['lon'], coord['lat'])
        xx -= left_top_x
        yy -= left_top_y

        search_space[xx - block_size // 2:xx + block_size // 2, yy - block_size // 2] = highlight
        search_space[xx - block_size // 2:xx + block_size // 2, yy + block_size // 2] = highlight
        search_space[xx - block_size // 2, yy - block_size // 2:yy + block_size // 2] = highlight
        search_space[xx + block_size // 2, yy - block_size // 2:yy + block_size // 2] = highlight
        coords.append(coord)
        final_results.append((coord['lon'], coord['lat']))

print("found {} windmills".format(found))

plt.rcParams['figure.figsize'] = [20, 10]
plt.imshow(search_space)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn import datasets, cluster, neighbors, metrics, tree, svm, mixture

#colors = np.array(['r', 'g', 'b', 'y', 'c', 't'])
colors = cm.rainbow(np.linspace(0, 1, 10))

def apply(X, algo):
    algo.fit(X)
    labels = algo.labels_.astype(np.int)
    plt.scatter(X[:, 0], X[:, 1], color=colors[labels])
    plt.show()
    return labels

df = pd.DataFrame(coords)
X = df.to_numpy()


#apply(X, cluster.KMeans(n_clusters=3))
apply(X, cluster.DBSCAN(eps=.011))
#apply(X, cluster.SpectralClustering(n_clusters=2, affinity="nearest_neighbors"))

In [ ]:
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import geopandas as gpd
import pandas as pd


list_lon = []
list_lat = []
list_geom = []
for final_coord in final_results:
    list_lon.append(final_coord[0])
    list_lat.append(final_coord[1])

# df_result = gpd.GeoDataFrame({
#                 'lon': pd.Series(list_lon, dtype='float'),
#                 'lat': pd.Series(list_lat, dtype='float'),
#     })
df_result = gpd.GeoDataFrame({
        'geometry': gpd.points_from_xy(list_lon, list_lat, crs="EPSG:4326").buffer(0.008)
})
df_result['geometry']
from shapely.ops import unary_union
cu = unary_union(df_result['geometry']);
df2_result = gpd.GeoDataFrame({
        'geometry': cu
})


# final_results
#
# df_result = gpd.GeoDataFrame({
#                 'lon': pd.Series(list_lon, dtype='float'),
#                 'lat': pd.Series(list_lat, dtype='float'),
#                 'geometry': pd.Series(list_geom, dtype='geometry')})
df2_result.crs = 'epsg:4326'
df2_result.to_file("result.shp")